# Agenter Quickstart

This notebook demonstrates the core features of Agenter - the backend-agnostic SDK for autonomous coding agents.

## Setup

```bash
# Install core agenter with all backends and adapters used in this notebook
pip install agenter[codex,claude-code,langgraph,pydantic-ai]

# Use your API keys for LLM providers
export ANTHROPIC_API_KEY="your-api-key"  # For anthropic-sdk and claude-code backends
export OPENAI_API_KEY="your-api-key"     # For codex backend
```

In [ ]:
import tempfile
from pathlib import Path

from agenter import AutonomousCodingAgent, CodingRequest, configure_logging

# Silence verbose debug logs for cleaner notebook output
configure_logging(level="WARNING")

# Use configure_logging(level="DEBUG") to see all internal logs

## 1. Basic Usage

The simplest way to use Agenter: give it a prompt and a directory.

In [ ]:
# Create a temporary workspace
workspace_1 = tempfile.mkdtemp(prefix="agenter_demo_")

agent = AutonomousCodingAgent()
result = await agent.execute(
    CodingRequest(
        prompt="Create a minimal Python function that calculates Fibonacci numbers.",
        cwd=workspace_1,
    )
)
result

**Status:** completed

**Summary:** Task completed successfully


**fibonacci.py**
```py
def fibonacci(n):
    """Calculate the nth Fibonacci number."""
    if n <= 1:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)


# Example usage
if __name__ == "__main__":
    # Test the function
    for i in range(10):
        print(f"fibonacci({i}) = {fibonacci(i)}")
```

## 2. Streaming Events

**Built-in display**: Use `verbosity=Verbosity.NORMAL` for Rich-formatted output.

**Custom event handling**: Process events exactly how you want (for example, for your custom UI or logging setup).

In [ ]:
from agenter import Verbosity
from agenter.data_models import CodingEventType
from agenter.data_models.messages import ToolCallMessage

workspace_2 = tempfile.mkdtemp(prefix="agenter_stream_")

agent = AutonomousCodingAgent()
request = CodingRequest(
    prompt="Create a calculator module with add, subtract, multiply, divide.",
    cwd=workspace_2,
)

tools_used = []
async for event in agent.stream_execute(request, verbosity=Verbosity.NORMAL):
    if event.type == CodingEventType.BACKEND_MESSAGE and isinstance(event.message, ToolCallMessage):
        tools_used.append(event.message.tool_name)

print(f"List of tool used: {tools_used}")

╔═══════════════════════════════════════════════════ Agenter ═══════════════════════════════════════════════════╗
║ Working directory: /var/folders/0f/1wtyzkj508nbh_hny1k3rrbm0000gn/T/agenter_stream_ya263zkw                   ║
║ Model: us.anthropic.claude-sonnet-4-20250514-v1:0                                                               ║
╚═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╝

▶ Iteration 1

╭─────────────────────────────────────────────────── 📝 Prompt ───────────────────────────────────────────────────╮
│ Create a calculator module with add, subtract, multiply, divide.                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 📨 Response (679 tokens) ────────────────────────────────────────────╮
│ I'll create a calculator module with the four basic arithmetic operations. Let me start by creating a Python    │
│ module file.                                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── 🔧 Tool ✓ ───────────────────────────────────────────────────╮
│ write_file calculator.py                                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────────── 🔧 Tool ✓ ───────────────────────────────────────────────────╮
│ write_file README.md                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 📨 Response (357 tokens) ────────────────────────────────────────────╮
│ Perfect! I've created a complete calculator module with the following files:                                    │
│                                                                                                                 │
│ ## Files Created:                                                                                               │
│                                                                                                                 │
│ 1. **`calculator.py`** - The main calculator module containing:                                                 │
│    - `add(x, y)` - Addition function                                                                            │
│    - `subtract(x, y)` - Subtraction function                                                                    │
│    - `multiply(x, y)` - Multiplication function                                                                 │
│    - `divide(x, y)` - Division function with zero-division error handling                                       │
│    - Built-in test cases that run when the module is executed directly                                          │
│                                                                                                                 │
│ 2. **`test_calculator.py`** - A comprehensive test file that                                                    │
│ ... (1,531 chars total)                                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── ✓ Validation ──────────────────────────────────────────────────╮
│ All checks passed                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╔════════════════════════════════════════════════════ Summary ════════════════════════════════════════════════════╗
║   Status:            ✅ Completed                                                                               ║
║   Iterations:        1                                                                                          ║
║   Tokens:            9,562                                                                                      ║
║   Duration:          23.3s                                                                                      ║
║   Files modified:    3                                                                                          ║
║                        • calculator.py                                                                          ║
║                        • test_calculator.py                                                                     ║
║                        • README.md                                                                              ║
╚═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╝

List of tool uses: ['write_file', 'write_file', 'write_file']


## 3. Structured Outputs

Get typed, validated outputs using Pydantic models.

In [ ]:
from pydantic import BaseModel


class CodeAnalysis(BaseModel):
    """Structured output for code analysis."""

    summary: str
    issues: list[str]
    suggestions: list[str]
    complexity_score: int  # 1-10


workspace_3 = tempfile.mkdtemp(prefix="agenter_structured_")

# Create a file to analyze
Path(workspace_3, "example.py").write_text("""
def process_data(data):
    result = []
    for item in data:
        if item > 0:
            for i in range(item):
                if i % 2 == 0:
                    result.append(i * item)
    return result
""")

agent = AutonomousCodingAgent()
result = await agent.execute(
    CodingRequest(
        prompt="Analyze the code in example.py for complexity and potential issues.",
        cwd=workspace_3,
        output_type=CodeAnalysis,  # With structured output
    )
)

analysis: CodeAnalysis = result.output

print(f"Summary: {analysis.summary}")
print(f"Complexity: {analysis.complexity_score}/10")
print(f"Issues: {analysis.issues}")
print(f"Suggestions: {analysis.suggestions}")

Summary: The code defines a function `process_data` that processes a list of numbers by generating sequences for positive values and appending calculated results. The function has moderate complexity due to nested loops and conditional logic.
Complexity: 6/10
Issues: ["No input validation - function will fail if 'data' is not iterable", 'No type hints provided, making the code less maintainable', "No docstring explaining the function's purpose and parameters", 'Potential performance issue with nested loops for large datasets', "Function name 'process_data' is too generic and doesn't describe what it actually does", 'No handling of edge cases like empty lists or non-numeric values']
Suggestions: ["Add input validation to check if 'data' is a list/iterable of numbers", 'Add type hints: def process_data(data: List[int]) -> List[int]', "Add a descriptive docstring explaining the function's behavior", 'Consider using list comprehension for better performance and readability', "Rename functi

## 4. Switching Backends

Switch between backends without changing your code.

In [ ]:
workspace_4 = tempfile.mkdtemp(prefix="agenter_backends_")

task = CodingRequest(
    prompt="Create a hello.py that prints 'Hello, World!'",
    cwd=workspace_4,
)

# Same task, different backends - each uses its own default model
for backend in ["anthropic-sdk", "codex"]:
    agent = AutonomousCodingAgent(backend=backend)
    result = await agent.execute(task)
    print(f"{backend}: {result.status} (${result.total_cost_usd:.4f})")

anthropic-sdk: CodingStatus.COMPLETED ($0.0085)
codex: CodingStatus.COMPLETED ($0.0004)


## 5. Framework Adapters

Adapters **subclass framework base classes** to set up tracing and all framework features automatically.

- **LangGraph**: `create_coding_node()` returns `RunnableLambda` → LangSmith tracing works when `LANGSMITH_API_KEY` is set
- **PydanticAI**: `CodingAgent` subclasses `pydantic_ai.Agent` → Logfire tracing works automatically

In [ ]:
# PydanticAI adapter - CodingAgent IS-A pydantic_ai.Agent
from agenter.adapters.pydantic_ai import CodingAgent

workspace_5 = tempfile.mkdtemp(prefix="agenter_pydantic_")

# CodingAgent subclasses pydantic_ai.Agent and overrides .run()
# Using codex backend (OpenAI models)
agent = CodingAgent(cwd=workspace_5, backend="codex")

result = await agent.run("Create a hello world script.")

print(f"Status: {result.status}")
print(f"Summary: {result.summary}")

Status: CodingStatus.COMPLETED
Summary: Task completed successfully


In [ ]:
# LangGraph adapter - returns RunnableLambda for LangSmith tracing
from agenter.adapters.langgraph import create_coding_node

workspace_6 = tempfile.mkdtemp(prefix="agenter_langgraph_")

# create_coding_node returns RunnableLambda (langchain base class)
# Using anthropic-sdk
coding_node = create_coding_node(cwd=workspace_6, backend="anthropic-sdk")

# Use ainvoke() - LangSmith traces automatically
state = {"prompt": "Create a utility function to format dates."}
result_state = await coding_node.ainvoke(state)

print(f"Status: {result_state['coding_result']['status']}")
print(f"Summary: {result_state['coding_result']['summary']}")

Status: CodingStatus.COMPLETED
Summary: Task completed successfully


## Cleanup

In [ ]:
import shutil

for ws in [workspace_1, workspace_2, workspace_3, workspace_4, workspace_5, workspace_6]:
    try:
        shutil.rmtree(ws)
        print(f"Dir '{ws}' cleaned up.")
    except Exception:
        pass

Cleaned up: /var/folders/0f/1wtyzkj508nbh_hny1k3rrbm0000gn/T/agenter_demo_p4mrwzzn
Cleaned up: /var/folders/0f/1wtyzkj508nbh_hny1k3rrbm0000gn/T/agenter_stream_ya263zkw
Cleaned up: /var/folders/0f/1wtyzkj508nbh_hny1k3rrbm0000gn/T/agenter_structured_1p1ogc1y
Cleaned up: /var/folders/0f/1wtyzkj508nbh_hny1k3rrbm0000gn/T/agenter_backends_ywlyivja
Cleaned up: /var/folders/0f/1wtyzkj508nbh_hny1k3rrbm0000gn/T/agenter_pydantic_nwp8eh6b
Cleaned up: /var/folders/0f/1wtyzkj508nbh_hny1k3rrbm0000gn/T/agenter_langgraph_tsyjtfh_
